In [46]:
# !pip install torchsummary

In [72]:
import torch
import os
import cv2
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.torch_version
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
from torchvision import transforms
from torchmetrics import Accuracy, Precision, Recall

import cv2
from joblib import Parallel, delayed

from torch.cuda.amp import GradScaler, autocast


In [73]:
import torch
print(torch.__version__)

2.3.1+cu121


In [74]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [75]:
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
main_path = '/kaggle/input/isic-2024-challenge' if iskaggle else 'data/isic-2024-challenge'

In [149]:
train_metadata_path = '/kaggle/input/isic-2024-challenge/train-metadata.csv' if iskaggle else 'data/isic-2024-challenge/train-metadata.csv'
test_metadata_path = '/kaggle/input/isic-2024-challenge/test-metadata.csv' if iskaggle else 'data/isic-2024-challenge/test-metadata.csv'

train_metadata_df = pd.read_csv(train_metadata_path)
test_metadata_df = pd.read_csv(test_metadata_path)

print(len(train_metadata_df))

C:\Users\abdfa\AppData\Local\Temp\ipykernel_18372\3275392473.py:4: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata_df = pd.read_csv(train_metadata_path)


401059


In [154]:
import sklearn.model_selection as train_test_split

train_size = 0.8
# Splitting the train dataset into positive and negative samples and saving them in separate dataframes
postive_samples = train_metadata_df[train_metadata_df['target'] == 1]
negative_samples = train_metadata_df[train_metadata_df['target'] == 0]

# TODO: Taking Sample of 1% of the data
# postive_samples = postive_samples.sample(frac=0.1)
negative_samples = negative_samples.sample(frac=5*postive_samples.shape[0]/negative_samples.shape[0])

print(f"Positive samples: {postive_samples.shape}")
print(f"Negative samples: {negative_samples.shape}")

Positive samples: (393, 55)
Negative samples: (1965, 55)


In [155]:

# Splitting each type of samples into train and validation sets
train_positive_samples, val_positive_samples = train_test_split.train_test_split(postive_samples,test_size=1-train_size)
train_negative_samples, val_negative_samples = train_test_split.train_test_split(negative_samples,test_size=1-train_size)
print(f"Train positive samples: {train_positive_samples.shape}")
print(f"Train negative samples: {train_negative_samples.shape}")
print(f"Val positive samples: {val_positive_samples.shape}")
print(f"Val negative samples: {val_negative_samples.shape}")


Train positive samples: (314, 55)
Train negative samples: (1572, 55)
Val positive samples: (79, 55)
Val negative samples: (393, 55)


In [156]:

# Concatenating the positive and negative samples to get the train and validation sets
train_metadata_df = pd.concat([train_positive_samples, train_negative_samples])
val_metadata_df = pd.concat([val_positive_samples, val_negative_samples])
print(f"Train samples: {train_metadata_df.shape}")
print(f"Val samples: {val_metadata_df.shape}")
print(f"Test samples: {test_metadata_df.shape}")


Train samples: (1886, 55)
Val samples: (472, 55)
Test samples: (3, 44)


In [157]:
import h5pickle as h5py
from io import BytesIO

train_hdf5_path = '/kaggle/input/isic-2024-challenge/train-image.hdf5' if iskaggle else 'data/isic-2024-challenge/train-image.hdf5'
test_hdf5_path = '/kaggle/input/isic-2024-challenge/test-image.hdf5' if iskaggle else 'data/isic-2024-challenge/test-image.hdf5'
train_image_path = '/kaggle/input/isic-2024-challenge/train-image/image' if iskaggle else 'data/isic-2024-challenge/train-image/image'

In [158]:
class CustomDataset(Dataset):
    def __init__(self, hdf5_file_path, metadata_df,target=None, transform=None):
        self.hdf5_file_path = hdf5_file_path
        self.hdf5_file = h5py.File(self.hdf5_file_path, 'r')
        self.metadata_df = metadata_df
        self.image_ids = metadata_df['isic_id']
        self.labels = target
        self.transform = transform

        self.mean_of_color_channels = None  # Initialize as None
        self.std_of_color_channels = None   # Initialize as None
        # self._calculate_stats()

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids.iloc[idx]
        image = np.array(Image.open(BytesIO(np.array(self.hdf5_file[image_id]))),dtype=np.float32)/255
        
        if self.transform:
            image = self.transform(image=image)
            image = image['image']

        if self.labels is not None:
            label = self.labels.iloc[idx]
            return image, label
        else:
            return image

   

In [166]:
# HyperParameters
dim = 224 
batch_size = 64

In [167]:
train_transform = A.Compose([
    A.Resize(height=dim, width=dim), #resize 
    A.OneOf([A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15),
             A.RandomBrightnessContrast() 
             ], p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    # !!One needs to change the mean and std values to appropriate ones for this dataset.!!
    A.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=1.0),
    ToTensorV2(),
])


test_transform = A.Compose([
    A.Resize(height=dim, width=dim), #resize 
    # !!One needs to change the mean and std values to appropriate ones for this dataset.!!
    A.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=1.0),
    ToTensorV2(),
])

In [168]:
train_dataset = CustomDataset(train_hdf5_path,train_metadata_df,target=train_metadata_df['target'],transform=train_transform)
# train_image_dataset = CustomDatasetImage(train_image_path,train_metadata_df,target=train_metadata_df['target'],transform=train_transform)
val_dataset = CustomDataset(train_hdf5_path,val_metadata_df,target=val_metadata_df['target'],transform=train_transform)
test_dataset = CustomDataset(test_hdf5_path,test_metadata_df,transform=test_transform)

In [169]:
n_w = 0
if iskaggle:
    n_w = 4
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True,num_workers=n_w)
val_loader = DataLoader(val_dataset,batch_size=batch_size,shuffle=True,num_workers=n_w)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False,num_workers=n_w)

In [170]:
from sklearn.metrics import roc_curve, auc

def calculate_partial_auc_by_tpr(y_true, y_scores, max_tpr=0.8):
    # Calculate the ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    
    # Limit the TPR to the specified minimum TPR
    mask = tpr >= max_tpr
    fpr, tpr = fpr[mask], tpr[mask]
    
    # Calculate the partial AUC
    partial_auc = auc(fpr, tpr)
    
    # Normalize the partial AUC to the range [0, 1]
    partial_auc /= max_tpr
    
    return partial_auc

In [175]:
class Model(nn.Module):
    def __init__(self,num_classes,device,dim = 32,num_epochs = 20,learning_rate = 0.001,early_stopping = False):
        super().__init__()
        self.num_of_classes = num_classes
        self.device = device
        self.dim = dim
        # Debugging
        self.DEBUG = True
        # Hyperparameters
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.early_stopping = early_stopping
        # History while Training
        self.model_loss_history = []

        self.model_train_acc_history = []
        self.model_train_pauc_history = []
        
        self.model_val_acc_history = []
        self.model_val_precision_history = []
        self.model_val_recall_history = []
        self.model_val_pauc_history = []
        
        self.model_lr_history = []

        # Model Attributes
        self.criterion = nn.BCEWithLogitsLoss()
        self.optimizer = None
        self.accuracy = Accuracy(task= 'binary', average='macro').to(self.device)
        self.precision = Precision(task= 'binary', average='macro').to(self.device)
        self.recall = Recall(task= 'binary', average='macro').to(self.device)
        # Feature Extraction
        self.feature_extract = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=7, padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten()
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(1152 , 200),  # Adjust the size based on the input image size
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(100, 1)

        )
        
    def forward(self, x):
        x = self.feature_extract(x)
        x = self.classifier(x)
        return x
    
    def predict(self, img):
        '''
        returns the predicted classes for the given images
        '''
        self.eval()
        with torch.no_grad():
            img = img.to(self.device)
            output = self(img)
            prob_outputs = torch.sigmoid(output)
            return prob_outputs
            
        

    
    def eval_val(self, data_loader):
        '''
        returns the accuracy, precision, recall , partial AUC, predicted probabilities and actual labels for the given data loader
        '''
        self.eval()
        y_prob_pred = []
        y_actual = []
        with torch.no_grad():
            for images, actuals in data_loader:
                
                images, actuals = images.to(self.device), actuals.to(self.device)
                # Get the model predictions
                output = self(images)
                # Get the probability outputs
                prob_outputs = torch.sigmoid(output)
                # Get the binary predictions
                batch_pred = (prob_outputs > 0.5).float()
                # Reshape the actuals
                actuals = actuals.unsqueeze(1).float()

                y_prob_pred.extend(prob_outputs.cpu().numpy())
                y_actual.extend(actuals.cpu().numpy())

                # Update the metrics
                self.accuracy(batch_pred, actuals)
                self.precision(batch_pred, actuals)
                self.recall(batch_pred, actuals)
        
        # Calculate the partial AUC
        partial_auc = calculate_partial_auc_by_tpr(y_true=y_actual, y_scores=y_prob_pred, max_tpr=0.8)


        return self.accuracy.compute(), self.precision.compute(), self.recall.compute(), partial_auc, y_prob_pred, y_actual
    
    def train_model(self, train_loader, val_loader):
        
        last_accuracy = -100
        self.optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        scaler = GradScaler()

        for epoch in range(self.num_epochs):
            self.train()
            running_loss = 0.0

            for i, (images, labels) in enumerate(train_loader):
                
                images, labels = images.to(self.device), labels.to(self.device)
                self.optimizer.zero_grad()
                with autocast():
                    outputs = self(images)
                    labels = labels.unsqueeze(1).float()
                    loss = self.criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(self.optimizer)
                scaler.update()

                running_loss += loss.item()
                if i%1000 == 0 and self.DEBUG:
                    print(" Step [{}/{}] Loss: {}".format(i, len(train_loader), loss.item()))
                    
            val_acc, val_precision, val_recall, val_pauc, _ , _ = self.eval_val(val_loader)
            train_acc, _, _, train_pauc, _, _ = self.eval_val(train_loader)

            self.model_loss_history.append(running_loss/len(train_loader))
            self.model_train_acc_history.append(train_acc.item())
            self.model_train_pauc_history.append(train_pauc)
            self.model_val_acc_history.append(val_acc.item())
            self.model_val_precision_history.append(val_precision.item())
            self.model_val_recall_history.append(val_recall.item())
            self.model_val_pauc_history.append(val_pauc)
            self.model_lr_history.append(self.optimizer.param_groups[0]['lr'])
            
            print(f'Epoch: {epoch+1}/{self.num_epochs}, Loss: {loss.item()},Train Acc: {train_acc}, Val Acc: {val_acc}, Val Precision: {val_precision}, Val Recall: {val_recall},Train PAUC: {train_pauc}, Val PAUC: {val_pauc}')
            
            if val_acc > last_accuracy:
                last_accuracy = val_acc
            elif self.early_stopping:
                break
        
        print('Finished Training')

    def plot_history(self):
        # making two plots one for loss and other for accuracy
        fig, axs = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle('Model Training History')
        axs[0, 0].plot(self.model_loss_history)
        axs[0, 0].set_title('Model Loss')
        axs[0, 0].set_xlabel('Epochs')
        axs[0, 0].set_ylabel('Loss')

        axs[0, 1].plot(self.model_train_acc_history, label='Train')
        axs[0, 1].plot(self.model_val_acc_history, label='Val')
        axs[0, 1].set_title('Model Accuracy')
        axs[0, 1].set_xlabel('Epochs')
        axs[0, 1].set_ylabel('Accuracy')
        axs[0, 1].legend()

        axs[1, 0].plot(self.model_val_precision_history)
        axs[1, 0].set_title('Model Precision')
        axs[1, 0].set_xlabel('Epochs')
        axs[1, 0].set_ylabel('Precision')
        
        axs[1, 1].plot(self.model_val_recall_history)
        axs[1, 1].set_title('Model Recall')
        axs[1, 1].set_xlabel('Epochs')
        axs[1, 1].set_ylabel('Recall')

        axs[0, 2].plot(self.model_lr_history)
        axs[0, 2].set_title('Learning Rate')
        axs[0, 2].set_xlabel('Epochs')
        axs[0, 2].set_ylabel('Learning Rate')
        
        axs[1, 2].plot(self.model_val_pauc_history)
        axs[1, 2].set_title('Model Partial AUC')
        axs[1, 2].set_xlabel('Epochs')
        axs[1, 2].set_ylabel('Partial AUC')
        

        plt.show()
    
    def save_model(self):
        torch.save(self.state_dict(),type(self).__name__+'.pth')

    def print_summary(self):
        summary(self, (3, self.dim, self.dim))

In [177]:
num_of_classes = 2
cnn = Model(num_classes=num_of_classes, 
            device=device, 
            dim=dim, 
            num_epochs=10, 
            learning_rate=0.001,
            early_stopping=False)
cnn.to(device)
cnn.print_summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 220, 220]           4,736
         MaxPool2d-2         [-1, 32, 110, 110]               0
            Conv2d-3         [-1, 64, 108, 108]          51,264
         MaxPool2d-4           [-1, 64, 54, 54]               0
            Conv2d-5          [-1, 128, 52, 52]         204,928
         MaxPool2d-6          [-1, 128, 26, 26]               0
            Conv2d-7           [-1, 64, 26, 26]          73,792
         MaxPool2d-8           [-1, 64, 13, 13]               0
            Conv2d-9           [-1, 32, 13, 13]          18,464
        MaxPool2d-10             [-1, 32, 6, 6]               0
          Flatten-11                 [-1, 1152]               0
           Linear-12                  [-1, 200]         230,600
             ReLU-13                  [-1, 200]               0
          Dropout-14                  [

In [178]:
cnn.train_model(train_loader=train_loader,val_loader=val_loader)

 Step [0/30] Loss: 0.6881775856018066
Epoch: 1/10, Loss: 0.2757985293865204,Train Acc: 0.8333333134651184, Val Acc: 0.8326271176338196, Val Precision: 0.0, Val Recall: 0.0,Train PAUC: 0.29389591538224663, Val PAUC: 0.21457387187167845
 Step [0/30] Loss: 0.39764153957366943
Epoch: 2/10, Loss: 0.3761991560459137,Train Acc: 0.8333333134651184, Val Acc: 0.8332155346870422, Val Precision: 0.0, Val Recall: 0.0,Train PAUC: 0.20232907489343777, Val PAUC: 0.1823243791670693
 Step [0/30] Loss: 0.44641023874282837
Epoch: 3/10, Loss: 0.5528662800788879,Train Acc: 0.8333333134651184, Val Acc: 0.8332690596580505, Val Precision: 0.0, Val Recall: 0.0,Train PAUC: 0.3303329767750929, Val PAUC: 0.21405047186523657
 Step [0/30] Loss: 0.4929148554801941
Epoch: 4/10, Loss: 0.579434871673584,Train Acc: 0.8333333134651184, Val Acc: 0.8332891464233398, Val Precision: 0.0, Val Recall: 0.0,Train PAUC: 0.28329919693359906, Val PAUC: 0.3359020195187941
 Step [0/30] Loss: 0.35139915347099304
Epoch: 5/10, Loss: 0.45

In [179]:
cnn_acc , cnn_precision, cnn_recall , cnn_pauc, y_pred, y_actual = cnn.eval_val(val_loader)
print(f"Accuracy: {cnn_pauc}, Precision: {cnn_precision}, Recall: {cnn_recall}")

Accuracy: 0.19436257931523176, Precision: 0.0, Recall: 0.0


In [180]:
# 0.5 threshold
y_pred = np.array(y_pred)
y_actual = np.array(y_actual).astype(int)
y_pred = (y_pred > 0.5).astype(int)

# output the incorrect predictions
incorrect_predictions = np.where(y_pred != y_actual)[0]
number_zero = np.sum(y_pred == 0)
number_one = np.sum(y_pred == 1)
number_zero, number_one

(472, 0)

In [181]:
print(f"Partial AUC: {cnn_pauc}")


Partial AUC: 0.19436257931523176


In [182]:
cnn.save_model()

In [183]:
y_prod_pred = []
for images in test_loader:
    images = images.to(device)
    prob_outputs = cnn.predict(images)
    y_prod_pred.extend(prob_outputs.cpu().numpy())

y_prod_pred = np.array(y_prod_pred)
y_prod_pred = y_prod_pred.flatten()
y_prod_pred

array([0.18550608, 0.2090997 , 0.1830071 ], dtype=float32)

In [184]:
submission_df = pd.DataFrame({'isic_id': test_metadata_df['isic_id'], 'target': y_prod_pred})
submission_df 

,isic_id,target
0,ISIC_0015657,0.185506
1,ISIC_0015729,0.209100
2,ISIC_0015740,0.183007


In [185]:
submission_df.to_csv('submission.csv', index=False)